# Темы семинара: отбор признаков и приложения

- Фильтрационные методы
- Оберточные методы
- Встроенные методы
- Нелинейные классификаторы
- Сохранение модели

In [4]:
import pandas as pd

In [5]:
data = pd.read_csv('Pokemon.csv')

In [6]:
data

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


Columns description (it's crucial!)


- #: ID for each pokemon
- Name: Name of each pokemon
- Type 1: Each pokemon has a type, this determines weakness/resistance to attacks
- Type 2: Some pokemon are dual type and have 2
- Total: sum of all stats that come after this, a general guide to how strong a pokemon is
- HP: hit points, or health, defines how much damage a pokemon can withstand before fainting
- Attack: the base modifier for normal attacks (eg. Scratch, Punch)
- Defense: the base damage resistance against normal attacks
- SP Atk: special attack, the base modifier for special attacks (e.g. fire blast, bubble beam)
- SP Def: the base damage resistance against special attacks
- Speed: determines which pokemon attacks first each round

In [7]:
# fillna and drop useless cols

display(data.isnull().sum())
data['Type 2'] = data['Type 2'].fillna('No 2nd type')

data.drop(columns=['#', 'Name'], inplace=True)

#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

In [8]:
X = data.drop(columns='Legendary')
y = data['Legendary'].astype('int')

In [10]:
y.value_counts(normalize=True)

0    0.91875
1    0.08125
Name: Legendary, dtype: float64

# Make some default pipeline

In [12]:
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from category_encoders.leave_one_out import LeaveOneOutEncoder
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
import sklearn

# !pip install category_encoders

In [13]:
# define cat_cols

cat_cols = ['Type 1', 'Type 2']

default_pipeline = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))]
)

In [14]:
cv_res1 = cross_validate(default_pipeline,
                        X,
                        y,
                        cv=5,
                        scoring='f1',
                        n_jobs=-1,
                        return_train_score=True
                       )

In [15]:
cv_res1

{'fit_time': array([0.14399505, 0.14599848, 0.14699697, 0.12300086, 0.06800127]),
 'score_time': array([0.02900171, 0.02699733, 0.02499843, 0.02499843, 0.02199888]),
 'test_score': array([0.5       , 0.72727273, 0.47619048, 0.38095238, 0.64864865]),
 'train_score': array([0.71287129, 0.56097561, 0.6744186 , 0.7311828 , 0.72727273])}

In [16]:
cv_res1['test_score'].mean()

0.5466128466128467

# Make pipeline more complicated

In [17]:
# difficult pipeline

pipe_dif = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('model_', SVC(kernel='linear'))]
)

In [18]:
cv_res2 = cross_validate(pipe_dif,
                        X,
                        y,
                        cv=5,
                        scoring='f1',
                        n_jobs=-1,
                        return_train_score=True
                       )

cv_res2

{'fit_time': array([0.13900018, 0.15400338, 0.1470015 , 0.22099948, 0.23900008]),
 'score_time': array([0.02400398, 0.02999854, 0.03000236, 0.02800035, 0.023     ]),
 'test_score': array([0.375     , 0.88888889, 0.5       , 0.66666667, 0.53658537]),
 'train_score': array([0.95145631, 0.89583333, 0.97142857, 0.96153846, 0.98076923])}

In [20]:
cv_res2['train_score'].mean()

0.9522051815498418

In [19]:
cv_res2['test_score'].mean()

0.5934281842818427

train_score - просто класс ! модель получилась сложная, только очевидно переобученная ...

согласны, узнали ?


# Introduce feature selectors

In [21]:
data_tr = pipe_dif[:-1]

In [22]:
data_tr

Pipeline(steps=[('cat_encoder_', LeaveOneOutEncoder(cols=['Type 1', 'Type 2'])),
                ('poly_featurizer_', PolynomialFeatures(degree=4)),
                ('scaler_', StandardScaler())])

In [23]:
X_tr = data_tr.fit_transform(X, y)
print(f'data shape after transformation is {X_tr.shape}')

data shape after transformation is (800, 1001)


c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


1k признаков - многовато, добавим в пайплайн селектор

## Фильтрационные методы

Суть таких методов в том, чтобы для каждого признака посчитать некоторую метрику "связи" с целевым признаком. И в результате оставить топ-K признаков согласно выбранной метрике.

В том числе на лекции обсуждались:

 - статистика хи-квадрат
 - метрика mutual information

In [24]:
from sklearn.feature_selection import f_classif, chi2, mutual_info_classif

In [28]:
k_best = 100

pipe = Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectKBest(score_func=mutual_info_classif, k=k_best)), 
    ('model_', SVC(kernel='linear'))]
)

In [29]:
cv_res = cross_validate(pipe, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res

c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_enc

{'fit_time': array([4.56791615, 5.04232812, 5.85330677, 4.4480052 , 4.7117455 ]),
 'score_time': array([0.01200581, 0.01699996, 0.01300287, 0.01400137, 0.01600385]),
 'test_score': array([0.26666667, 0.60869565, 0.41666667, 0.36363636, 0.57142857]),
 'train_score': array([0.82      , 0.66666667, 0.79591837, 0.75862069, 0.88235294])}

In [30]:
# k best нужно подбирать

cv_res['test_score'].mean()

0.44541878411443625

## Жадный метод отбора

In [31]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

In [32]:
k_best = 50

rfe = RFE(LogisticRegression(max_iter=1000), n_features_to_select=k_best, step=30)

In [33]:
X_tr.shape

(800, 1001)

In [34]:
res = rfe.fit_transform(X_tr, y)
display(res.shape)
res

(800, 50)

array([[-0.98555744, -0.53089061, -0.93734374, ..., -0.40535388,
        -0.52135831, -0.81966779],
       [-0.48479877, -0.53089061, -0.84621022, ..., -0.16466941,
        -0.18003271, -0.81698899],
       [ 0.42451538, -0.53089061, -0.68419506, ...,  0.45023305,
         0.73602435, -0.81341726],
       ...,
       [-0.16049792, -0.53089061,  1.16740671, ...,  3.97831918,
         2.1076934 ,  4.18343684],
       [-0.16049792, -0.53089061,  1.16740671, ...,  6.06069716,
         2.51608978,  4.18343684],
       [ 1.36562373, -0.53089061,  1.16740671, ...,  1.4680924 ,
         0.19750623,  2.6404483 ]])

In [38]:
pipe_rfe = Pipeline([    
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', RFE(LogisticRegression(max_iter=1000),
                      n_features_to_select=30,
                      step=10
                     )), 
    ('model_', SVC(kernel='linear'))])

In [39]:
cv_res3 = cross_validate(pipe_rfe, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res3

c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_enc

{'fit_time': array([18.53746104, 22.14069128, 17.1071074 , 17.01407051, 18.47805548]),
 'score_time': array([0.02500129, 0.03399396, 0.02699876, 0.02200127, 0.04100156]),
 'test_score': array([0.47058824, 0.92307692, 0.71428571, 0.58333333, 0.6875    ]),
 'train_score': array([0.91262136, 0.80434783, 0.95327103, 0.89361702, 0.92      ])}

In [40]:
cv_res3['test_score'].mean()

0.6757568411980176

## С помощью L1 регуляризации

In [41]:
from sklearn.feature_selection import SelectFromModel

In [42]:
sel = SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), threshold=1e-5)

In [43]:
# пример

res = sel.fit_transform(X_tr, y)
display(res.shape)
res

(800, 48)

array([[-1.20562657, -0.94218651, -0.7732015 , ..., -0.52135831,
        -0.72668962, -0.81966779],
       [-1.20562657, -0.94218651, -0.33767384, ..., -0.18003271,
        -0.70651232, -0.81698899],
       [-1.20562657, -0.94218651,  0.6523422 , ...,  0.73602435,
        -0.6731154 , -0.81341726],
       ...,
       [-1.20562657, -0.94218651,  0.80566326, ...,  2.1076934 ,
         4.87819432,  4.18343684],
       [-1.20562657, -0.94218651,  1.06849938, ...,  2.51608978,
         4.87819432,  4.18343684],
       [-1.20562657, -0.94218651,  2.2512619 , ...,  0.19750623,
         1.93926564,  2.6404483 ]])

In [44]:
pipe_lasso =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), 
                                  threshold=1e-5)), 
    ('model_', SVC(kernel='linear'))])

In [45]:
cv_res4 = cross_validate(pipe_lasso, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res4

c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_enc

{'fit_time': array([0.15653229, 0.17102098, 0.14936399, 0.14299631, 0.15499997]),
 'score_time': array([0.01003456, 0.01003432, 0.01496434, 0.01303506, 0.01400113]),
 'test_score': array([0.44444444, 0.88      , 0.68965517, 0.60869565, 0.66666667]),
 'train_score': array([0.92307692, 0.875     , 0.95238095, 0.93877551, 0.92929293])}

In [46]:
cv_res4['test_score'].mean()

0.6578923871397634

# Нелинейные классификаторы

- SVM с ядром
- Наивный байесовский классификатор
- Метод k ближайших соседей

In [50]:
pipe_lasso2 =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), 
                                  threshold=1e-5)), 
    ('model_', SVC(kernel='poly'))])

In [51]:
cv_res5 = cross_validate(pipe_lasso2, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res5

c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_enc

{'fit_time': array([0.15363836, 0.16796732, 0.15197659, 0.16809916, 0.18603945]),
 'score_time': array([0.01103783, 0.01204371, 0.01299667, 0.01296449, 0.01607919]),
 'test_score': array([0.55555556, 0.7826087 , 0.56      , 0.45454545, 0.68421053]),
 'train_score': array([0.79569892, 0.80898876, 0.9       , 0.91836735, 0.88659794])}

In [52]:
cv_res5['test_score'].mean()

0.6073840464137946

Обучите наивный байесовский классификатор и метод k ближайших соседей вместо SVM в пайплайне выше. 

In [53]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB

In [88]:
pipe_lasso3 =  Pipeline([
    ('cat_encoder_', LeaveOneOutEncoder(cols=cat_cols)),
    ('poly_featurizer_', PolynomialFeatures(degree=4)),
    ('scaler_', StandardScaler()),
    ('selector_', SelectFromModel(LogisticRegression(penalty='l1', max_iter=1000, solver='liblinear'), 
                                  threshold=1e-5)), 
    ('model_', GaussianNB())])

In [89]:
cv_res6 = cross_validate(pipe_lasso3, X, y, cv=5, scoring='f1', return_train_score=True)
cv_res6['test_score'].mean()

c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):
c:\programdata\miniconda3\lib\site-packages\category_enc

0.49630038362918183

С помощью GridSearch подберите гиперпараметр KNN (число соседей) внутри пайплайна.

In [82]:
pipe_pre = pipe_lasso3[:-1]
X_new = pipe_pre.fit_transform(X,y)

c:\programdata\miniconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [83]:
from sklearn.model_selection import GridSearchCV
import numpy as np

params = {'n_neighbors' : np.arange(2, 10, 2)}
model = KNeighborsClassifier()

gs = GridSearchCV(model, params, cv=5, scoring='f1')
gs.fit(X_new, y)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([2, 4, 6, 8])}, scoring='f1')

In [85]:
gs.best_score_, gs.best_estimator_

(0.4222059838428125, KNeighborsClassifier(n_neighbors=2))

# Сохранение и загрузка модели

In [90]:
import pickle

Обучим лучшую модель на всех данных и сохраним её в файл.

In [91]:
#your code here

model = gs.best_estimator_

Сохранение модели в файл

In [92]:
filename = 'best_model.sav'
pickle.dump(model, open(filename, 'wb'))

Загрузка модели из файла

In [93]:
loaded_model = pickle.load(open(filename, 'rb'))

In [94]:
loaded_model

KNeighborsClassifier(n_neighbors=2)

Второй вариант

In [2]:
import joblib

filename = 'best_model.sav'
joblib.dump(model, filename)
 
loaded_model = joblib.load(filename)

Почитать подробнее про сохранение модели в файл и загрузку из файла тут: https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/